## WORKING WITH LANGCHAIN
This workbook demonstrates usage of `Langchain` - framework that supports building applications with AI tools.

In [97]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import datetime as dt
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus
from pydantic import BaseModel, Field
from langchain.agents.structured_output import ToolStrategy, ProviderStrategy
from pathlib import Path

In [2]:
#.env path
env_path = Path(__file__).resolve().parent.parent.parent / '.env'
load_dotenv(dotenv_path=env_path)

# fetch API key
openai_api_key = os.getenv("OPENAI_API_KEY")


# create agent using Sonar-pro model and configure other model related params
model = ChatOpenAI(
    model='gpt-5.2',
    api_key=openai_api_key,
    temperature=0.3, # higher number indicates more randomness in the model's output
    max_tokens=500, #defines the no of words in the model's response
    timeout=30 # max time in sec to wait for model's response
)

### WIKI SEARCH USING TOOL
Here we create an agent that searches wikipedia for a particular query say name of a famous personality to fetch information as per user query.

In [10]:
from langchain.tools import tool
import wikipedia

@tool
def get_wikipedia_information(query: str):
    '''
    Function that fetches information from wikipedia about a well known personality.
    params: query: Name of a person.
    '''
    result = ""

    try:
        page = wikipedia.page(title=query,auto_suggest=True)
        result = wikipedia.summary(page.title,sentences=3)
        return result
    except wikipedia.DisambiguationError as e:
        return f"Failed to load page due to {e}. Please be more specific."
    except wikipedia.PageError as e:
        return f"Failed to load page due to {e}."

In [15]:
@tool
def math_tool(expression: str):
    '''
    Function that computes age basis year
    params: expression -> birth year
    '''
    return str(eval(expression))

In [16]:
@tool
def compute_age_diff_basis_dates(expression: str):
    '''
    Function that computes age in years now.
    params: expression: birth date
    '''

    birth_date = dt.datetime.strptime(expression, "%Y-%m-%d")
    today = dt.date.today()

    result = (today.year-birth_date.year)-((today.month,today.day)<(birth_date.month,birth_date.day))

    return result

### USING STRUCTURED OUTPUT
In order to get response from the **LLM** in a format desired we use **Pydantic** validations.

In [19]:
from pydantic import BaseModel, Field

class AgeInfo(BaseModel):
    '''
    Age information for a person.
    '''
    age: str = Field(description="Age information in years as of the current date.")

In [20]:
# create agent = model + tool
agent = create_agent(
    model=model,
    tools=[get_wikipedia_information,math_tool,compute_age_diff_basis_dates],
    response_format=AgeInfo
)

In [21]:
# run agent with the user query
response = agent.invoke({"messages":[{"role":"user","content":'When was elon musk born? what is his age  as of today?'}]})
print(response['structured_response'])

age='Elon Musk was born on June 28, 1971. As of today (2026-02-01), he is 54 years old.'


### BUILD A SEMANTIC SEARCH ENGINE WITH LANGCHAIN
The below section demonstrates usage of `Langchain` to setup a search engine over a PDF document which will help us to do the following, 
1. Extract passages in the PDF similar to input query. 

We build the architecture across 2 fronts namely, 
1. Data ingestion. 
2. Question/Query ansering

In [22]:
from langchain_community.document_loaders import PyPDFLoader

In [25]:
os.getcwd()

'/Users/soumyadipsikdar/Desktop/LLMs'

`PyPDFLoader` loads one **document** object per PDF page. Since **document** object comes with *page_content* & *metadata** properties this allows us to pull content & metadata which usually consists of the document source for each page.

In [26]:
file_path = 'data/nke-10k-2023.pdf'

# setup loader to load pdf
pdf_loader = PyPDFLoader(file_path=file_path)
docs = pdf_loader.load()

In [27]:
# display content and metadata of first page
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FO

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


In [28]:
# display number of pages in the document
print(f'No of pages in the document is: {len(docs)}')

No of pages in the document is: 107


#### TEXT-SPLITTERS
`Text-splitter` splits a document into chunks of a particular size. By default `RecursiveCharacterTextSplitter` is used which uses *['\n\n','\n',' ','']* to split texts while balancing context and chunk size i.e. it tries to split at paragraphs and then at sentences and then at words before finally splitting at each character. 

Using `text-splitter` ensures individual chunk is retrievable.

In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200,add_start_index=True)
all_splits = text_splitter.split_documents(docs)

print(f"No of chunks/split documents post text splitting is: {len(all_splits)}")

No of chunks/split documents post text splitting is: 516


In [30]:
print(all_splits[0])

page_content='Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE FISCAL YEAR ENDED MAY 31, 2023
OR
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE TRANSITION PERIOD FROM                         TO                         .
Commission File No. 1-10635
NIKE, Inc.
(Exact name of Registrant as specified in its charter)
Oregon 93-0584541
(State or other jurisdiction of incorporation) (IRS Employer Identification No.)
One Bowerman Drive, Beaverton, Oregon 97005-6453
(Address of principal executive offices and zip code)
(503) 671-6453
(Registrant's telephone number, including area code)
SECURITIES REGISTERED PURSUANT TO SECTION 12(B) OF THE ACT:
Class B Common Stock NKE New York Stock Exchange
(Title of each class) (Trading symbol) (Name of each exchange on which registered)' metadata={'pro

#### EMBEDDINGS
Next we convert text in each chunk into fixed size vectors using `embedding`. This helps to match embedded query semantically to the right text stored in vector database.

In [33]:
# setup embedding model from OpenAI
embedding = OpenAIEmbeddings(model="text-embedding-3-large",api_key=openai_api_key)

# vectorize the first 2 chunks/split documents
vector_1 = embedding.embed_query(all_splits[0].page_content)
vector_2 = embedding.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated fixed size vectors are of length: {len(vector_1)}")

Generated fixed size vectors are of length: 3072


In [34]:
# display vector
vector_1[:10]

[0.011540832929313183,
 -0.018770398572087288,
 0.00016181038517970592,
 0.005937446374446154,
 0.02033354714512825,
 -0.040011581033468246,
 -0.0075005958788096905,
 0.04122176021337509,
 -0.008786411955952644,
 0.060004767030477524]

#### VECTOR DATABASE/ VECTOR STORE
An efficient database that stores texts/document in the form of numerical fixed-size vectors. Often setup with an `embedding` any text or document added to it are stored in the form of vector. 

Here we use a lightweight `InMemoryVectorStore` object. 

The advantage of vector store is the following, 
1. Stores texts that are similar in semantic meaning together. 
2. This makes indexing documents/chunks easier without having to go through all paragraphs one by one and by using similarity metrics such as *cosine similarity*, *Euclidian distance* etc.

In [35]:
from langchain_core.vectorstores import InMemoryVectorStore

# setup InMemoryvectorstore object
vector_store = InMemoryVectorStore(embedding=embedding)

In [36]:
# index documents i.e. add documents into the vector database
ids = vector_store.add_documents(documents=all_splits)

#### QUERING
Now that the `vector database` has been added with chunks in the form of numerical vectors we can query it. 

`Quering` usually consists of the following steps, 
1. Each user query gets converted to a numerical vector of the same size as are the vectors in the database. 
2. Each query then is matched to the vectors using metrics such as *cosine similarity* etc. 
3. For each query the chunk with the most match is retrived by the system and answer is then drawn from it. 

For *similarity search* one can retrieve results with/without similarity scores.

In [38]:
# return documents based on similarity to a string query
results = vector_store.similarity_search_with_score("How many distribution centers does Nike have in the US?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.7455320815844053

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'producer': 'EDGRpdf Ser

In [41]:
# below we embed query first and then fetch results with similarity scores
query_embedded = embedding.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_with_score_by_vector(query_embedded)
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.7222294797453574

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
• Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
• Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
• Unfavorable changes in net foreign currency exchange rates, including hedges; and
• Lower off-price margin, on a wholesale equivalent basis.
Thi

#### RETRIEVERS
They are `runnables` that take a query, fetches & displays relevant texts as per query. We can either create a custom `retriver` or use the same using `vector store`. 

Below shows an implementation using `vector store`.

In [43]:
# create retriever from vector store
retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = {'k':1}
)

In [44]:
# pass queries to retriever & fetch results
result = retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ]
)

In [51]:
result[1][0].page_content

'Table of Contents\nPART I\nITEM 1. BUSINESS\nGENERAL\nNIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"\n"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.\nOur principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is\nthe largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores\nand sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales'

### SQL AGENT
Here we create an *SQL Agent* that considers user query converts to a database specific query and pass it to the backend database which in this case is a retail industry database named `Atliq T-shirt` in *MySQL* to extract necessary information.

In [52]:
# SQLDatabase is a wrapper that provides a simple interface to run sql queries
from langchain_community.utilities import SQLDatabase

In [59]:
# fetch necessary DB credentials
load_dotenv('.env')
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
localhost = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')

db_name = 'atliq_tshirts'

# create DB URI with above credentials
db_uri = f'mysql+mysqlconnector://{user}:{password}@{localhost}:{port}/{db_name}'

In [60]:
# setup connection to MYSQL database Atliq_tshirt using above URI
db = SQLDatabase.from_uri(db_uri)

#### FETCH DATABSE RELATED INFORMATION
With connection established we can now fetch database related information such as, 
1. Tables 
2. Columns present 
3. Database engine etc.

In [63]:
print(f'Database engine used is: {db.dialect}\n')
print(f'Available tables are: {db.get_usable_table_names()}\n')

Database engine used is: mysql

Available tables are: ['discounts', 't_shirts']



#### TOOLS FOR DATABASE INTERACTION
Next we setup `tools` that help us interact with the database to do the following, 
1. Fetch list of tables present. 
2. Check on query written to be executed to pull information from database. 
3. fetch data from the database using query etc.

In [64]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [65]:
# setup toolkit for db interaction
mysql_toolkit = SQLDatabaseToolkit(db=db,llm=model)

In [66]:
# View the different tools present with a corresponding short description of it.
tools = mysql_toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



#### CREATE A DETAILED SYSTEM PROMPT
We aim to setup the *SQL Agent* to behave like an analyst hence we provide it such a descriptive prompt. 

Notice the line `Never query for all columns under a table but only related ones` which says it pick and perform aggregation only on the columns that matter related to the user query. Also LLM's operate on a fixed token limit, by chosing to display all columns the token limit could overpass thereby crashing operation.

Also since the LLM comes up with it's own query prior to executing it to fetch results from the database it might be that the query turns up incorrect. This makes it important to double-check the query and re-write it if necessary before execution, hence the prompt `ouble check your query before executing`.

In [118]:
# setup a detailed system prompt to customise agent behaviour.
system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. DO NOT skip this step.

NEVER expose internal identifiers such as IDs, primary keys, or surrogate keys. ALWAYS return human-readable attributes instead.

NEVER show texts wrapped with ** around. ALWAYS display clean texts in response.

NOTE that the price column in t_shirts table has the unit of Rs.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=5,
)

#### SETUP AND RUN THE AGENT
Next we execute the agent with queries to see how well it answers.

In [114]:
query = "How many t-shirts do we have for Nike in extra-small and white color?"

In [82]:
class Output(BaseModel):
    answer : str = Field(description='Final answer to user query')

In [119]:
# setup agent
sql_agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    response_format=ProviderStrategy(Output)
)

In [115]:
response = sql_agent.invoke({"messages":[{"role":"user","content":query}]})
print(response['structured_response'])

answer='We have 12 Nike t-shirts in extra-small (XS) and white color.'


In [116]:
query = "How many is the price of inventory for all small sized t-shirts?"

In [120]:
response = sql_agent.invoke({"messages":[{"role":"user","content":query}]})
print(response['structured_response'])

answer='The total price of inventory for all small (S) sized t-shirts is Rs 16,437.'


In [126]:
query = "If we sell all the Levi's t-shirts today with discounts applied how much revenue would that generate?"

for event in sql_agent.stream({"messages":[{"role":"user","content":query}]},stream_mode="values"):
    last_message = event['messages'][-1].pretty_print()

================================ Human Message =================================

If we sell all the Levi's t-shirts today with discounts applied how much revenue would that generate?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_j4bXatay6K6H5VyPWxY28vQc)
 Call ID: call_j4bXatay6K6H5VyPWxY28vQc
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

discounts, t_shirts
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_NFhS7bSwVWtEz3AyUhTsQ3I2)
 Call ID: call_NFhS7bSwVWtEz3AyUhTsQ3I2
  Args:
    table_names: t_shirts, discounts
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (dis

In [127]:
query = "How many white color Levi's t shirts do we have available?"

response = sql_agent.invoke({"messages":[{"role":"user","content":query}]})
print(response['structured_response'])

answer='We have 230 white color Levi’s t-shirts available in stock.'


#### FEW-SHOT EXAMPLES
`FEW SHOT EXAMPLES` are a way to improve LLM performance by coming up with the right set of SQL queries everytime a user asks a question. 

Usually `FEW SHOT EXMAPLES` are included in the system prompt in the form, 

User: Question
SQL : sql query that answers the question. 

**A good system prompt would be,**

You are a SQL agent for a retail apparel database.

Follow these rules:
- Never use SELECT *
- Only select columns required to answer the question
- Use LEFT JOIN when optional data (e.g., discounts) may be missing
- Do not expose internal IDs in the final answer
- Prefer aggregation in SQL, not post-processing

Examples:

User: What is the total revenue for Levi brand? 

SQL:
SELECT
  SUM(ts.stock_quantity * ts.price * (1 - COALESCE(d.pct_discount, 0)/100)) AS revenue
FROM t_shirts ts
LEFT JOIN discounts d ON d.t_shirt_id = ts.t_shirt_id
WHERE ts.brand = 'Levi';

User: Which T-shirt has the lowest discount? 

SQL:
SELECT
  ts.brand,
  ts.color,
  ts.size,
  d.pct_discount
FROM t_shirts ts
JOIN discounts d ON d.t_shirt_id = ts.t_shirt_id
ORDER BY d.pct_discount ASC
LIMIT 1;
